## written by : Yoel Ross
## email: yoel.ross@gmail.com

## useful numpy functions:
 * where
 * take
 * argmin
 * logical_*
 * transpose
 * vstack, hstack, dstack
 * np.newaxis
 
## working examples:
 * mapping mapping pixel values, three possible implementations
 * iou calculation, three possible methods

In [1]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import cv2
from ipywidgets import interact, IntSlider
import os

## functions to know

In [ ]:
# return all indices of values that match a value
x = np.eye(3)
np.where(x == 1)

In [ ]:
# construct a new array from an existing one using indices
x = np.eye(5)
np.take(x, [0, 1], axis=0) # take rows
np.take(x, [0, 1], axis=1) # take columns

In [ ]:
# get the index of the minimum value
x = np.linspace(0, np.pi)
print(x[np.argmin(x)], x[np.argmax(x)])

In [ ]:
# can be applied across an axis as well!
x = np.eye(5)
print(np.argmax(x, axis=0))

## List vs. Vector operations
Applying a linear function y = ax + b to a one dimensional array or list:

In [2]:
list_size = 100000
py_list = [i for i in range(list_size)]
np_vect = np.array(py_list)

In [3]:
def apply_linear_func_py(py_list, a, b):
    return [a*i+b for i in py_list]

def apply_linear_func_np(vec, a ,b):
    return a*vec + b

In [4]:
%timeit apply_linear_func_py(py_list, 4, 6)

17.8 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit apply_linear_func_np(np_vect, 4, 6)

206 µs ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


How does this varry as the list gets larger:

In [6]:
python_list_time = []
numpy_vect_time = []
convrsion_time = []
for size in tqdm(range(100, 100000, 1000)):
    # create the list and vector
    py_list = [i for i in range(size)]
    np_vect = np.array(py_list)
    
    # apply function to python list
    t0 = time.time()
    apply_linear_func_py(py_list, 4, 6)
    t1 = time.time()
    python_list_time.append(t1-t0)
    
    # apply function to numpy vector
    t0 = time.time()
    apply_linear_func_np(np_vect, 4, 6)
    t1 = time.time()
    numpy_vect_time.append(t1-t0)
    
    # time conversion from list to ndarray
    t0 = time.time()
    temp_array = np.array(py_list)
    t1 = time.time()
    convrsion_time.append(t1-t0)
    

100%|██████████| 100/100 [00:05<00:00,  9.03it/s]


In [7]:
plt.figure(1)
plt.plot(python_list_time, c='b', label="python list")
plt.plot(numpy_vect_time, c='r', label="numpy array")
plt.legend()
plt.xlabel("list/vector size")
plt.ylabel("time, ms")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'time, ms')

How about elementwise binary operations?

In [ ]:
# setup some lists
l1 = list(range(1000))
l2 = list(range(1000))
%timeit [a+b for (a,b) in zip(l1, l2)]

In [ ]:
a1 = np.array(l1)
a2 = np.array(l2)
%timeit a1+a2

## Converting lists to arrays
Converting python data structures to numpy is rather costly. Converting back and forth should be avoided. If you are doing some heavy lifting, try and use numpy arrays only.

Look at the running time of converting a list to an ndarray relative to applying a linear function to the list:

In [ ]:
plt.figure(2)
plt.plot(python_list_time, c='b', label="python list linear op")
plt.plot(convrsion_time, c='r', label="constructing ndarray from list")
plt.legend()
plt.xlabel("list size")
plt.ylabel("time, ms")

If you can't, and you are carrying out a number of seperate operations on the list, it might be worth it:

In [ ]:
plt.figure(3)
plt.plot(convrsion_time, c='k', label="conversion time")
for i in range(1,5):
    plt.plot([i*t for t in python_list_time], label=f"num ops: {i}")
plt.legend()

## lets take a look at reductions and order statistics 
assuming a 2D matrix, or a list of list, how fast can we reduce across rows? Across columns?

In [ ]:
row_sum_np = []
row_sum_py = []

In [ ]:
for size in tqdm(range(10, 10000, 1000)):
    rand_mat = [[np.random.randint(0, 256) for v in range(size)] for v2 in range(size)]
    rand_arr = np.array(rand_mat)
    
    t0 = time.time()
    sum_list = [sum(l) for l in rand_mat]
    row_sum_py.append(time.time()-t0)
    
    t0 = time.time()
    sum_arr = np.sum(rand_arr, axis=0)
    row_sum_np.append(time.time()-t0)

plt.figure(4)
plt.plot(row_sum_py, c='r', label="python time")
plt.plot(row_sum_np, c='b', label="numpy time")
plt.xlabel("matrix dim")
plt.ylabel("run time")
plt.show()

## Broadcasting, or how to move your loops into C  
Using for loops in python can often be avoided by using vectorized operations, and by a process called broadcasting.
The term broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

The most simple case of broadcasting:

In [ ]:
a = np.array([1,2,3])
b = np.array([4])
c = a + b
print(c)

the scalar b is "broadcasted" to match the dimensions of a. Logically, think of b being copied along its only dimension to match the size of a.

This same behavior can be applied to arrays with multiple dimension, and the essential behavior remains the same. Numpy logically duplicated the array along the mismatched dimension.

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when:
   1. The dimensions are equal
   2. one of the matching directions equal one.

The array with the dimension that equlas 1 is broadcasted to match the second. Let's take a look at the first example:

In [ ]:
print(f"shape of a: {a.shape}, shape of b: {b.shape}, shape of c after broadcasting -> {c.shape}")      

For an extensive explaination of the broadcasting mechanic, see the [numpy broadcasting manual](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).

## Case study: mapping image pixels. 
let's consider a case where we want to implement a mapping of our image pixels to a small set of pixel values. We can choose which pixel to map to using the euclidian distance between the original pixel and the set of target pixels, choosing the target with the minimum distance.

#### The setup:
   * We have a list of 256 triplets, each repersenting a pixel color we can map to.
   * We have an input image with shape (200, 200, 3), we want to map each image pixel to the closest color given in the list.

Example is based on the native python implementation found at [this repo.](https://github.com/magarcia/python-x256/blob/master/x256/x256.py)

*source: Lisa Alali*
#### First Attempt: Native Python
The natural python implementation would use:    
   1. list of colors
   2. distance function: calculate distance between two pixels  
   3. get mapped pixel: calculates distances between a pixel and all other 256 colors, returns the minimum  
   4. map image: iterates through each image pixel and maps it, returning a mapped image.  
   

In [ ]:
from math import sqrt
def __distance(a, b):
    """
    euclidian distance between to 3-tuples.
    """
    x = (a[0] - b[0]) ** 2
    y = (a[1] - b[1]) ** 2
    z = (a[2] - b[2]) ** 2
    return sqrt(x + y + z)


def from_rgb(r, g=None, b=None):
    """
    Return the nearest xterm 256 color code from rgb input.
    """
    c = r if isinstance(r, list) else [r, g, b]
    best = {}

    for index, item in enumerate(colors):
        d = __distance(item, c)
        if(not best or d <= best['distance']):
            best = {'distance': d, 'index': index}

    if 'index' in best:
        return best['index']
    else:
        return 1

def map_image_pixels(image):
    """
    replace all image pixels with the closest xterm color
    :param image: ndarray image
    :return: ndarray image
    """
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            new_color = from_rgb(list(image[i, j]))
            image[i, j] = np.array(colors[new_color])
    return image

#### The promised list of colors:

In [ ]:
def __hex2rgb(hexa):
    r = int(hexa[0:2], 16)
    g = int(hexa[2:4], 16)
    b = int(hexa[4:6], 16)
    return [r, g, b]

colors = list(map(__hex2rgb, ["000000", "800000","008000" ,"808000" ,"000080" , "800080", "008080", "c0c0c0", "808080", "ff0000", "00ff00", "ffff00", "0000ff", "ff00ff",  
                              "005f00", "005f5f", "005f87", "005faf", "005fd7", "005fff", "008700", "00875f", "008787", "0087af", "0087d7", "ffffff", "000000", "00005f", 
                              "0087ff", "00af00", "00af5f", "00af87", "00afaf", "00afd7", "00afff", "00d700", "00d75f", "00d787", "00d7af", "000087", "0000af", "0000d7", 
                              "00d7d7", "00d7ff", "00ff00", "00ff5f", "00ff87",  "00ffaf","00ffd7", "00ffff", "5f0000", "5f005f", "5f0087", "5f00af", "5f00d7", "5f00ff",
                              "5f5f00", "5f5f5f", "5f5f87", "5f5faf", "5f5fd7", "5f5fff", "5f8700", "5f875f", "5f8787", "5f87af", "5f87d7", "0000ff", "00ffff",
                              "5f87ff", "5faf00", "5faf5f", "5faf87", "5fafaf", "5fafd7", "5fafff", "5fd700", "5fd75f", "5fd787", "5fd7af",
                              "5fd7d7", "5fd7ff", "5fff00", "5fff5f", "5fff87", "5fffaf", "5fffd7", "5fffff", "870000", "87005f", "870087",
                              "8700af", "8700d7", "8700ff", "875f00", "875f5f", "875f87", "875faf", "875fd7", "875fff", "878700", "87875f",
                              "878787", "8787af", "8787d7", "8787ff", "87af00", "87af5f", "87af87", "87afaf", "87afd7", "87afff", "87d700",
                              "87d75f", "87d787", "87d7af", "87d7d7", "87d7ff", "87ff00", "87ff5f", "87ff87", "87ffaf", "87ffd7", "87ffff",
                              "af0000", "af005f", "af0087", "af00af", "af00d7", "af00ff", "af5f00", "af5f5f", "af5f87", "af5faf", "af5fd7",
                              "af5fff", "af8700", "af875f", "af8787", "af87af", "af87d7", "af87ff", "afaf00", "afaf5f", "afaf87", "afafaf",
                              "afafd7", "afafff", "afd700", "afd75f", "afd787", "afd7af", "afd7d7", "afd7ff", "afff00", "afff5f", "afff87",
                              "afffaf", "afffd7", "afffff", "d70000", "d7005f", "d70087", "d700af", "d700d7", "d700ff", "d75f00", "d75f5f", "d75f87", "d75faf", "d75fd7",
                              "d75fff", "d78700", "d7875f", "d78787", "d787af", "d787d7", "d787ff", "d7af00", "d7af5f", "d7af87", "d7afaf", "d7afd7", "d7afff", "d7d700",
                              "d7d75f", "d7d787", "d7d7af", "d7d7d7", "d7d7ff", "d7ff00", "d7ff5f", "d7ff87", "d7ffaf", "d7ffd7", "d7ffff", "ff0000", "ff005f", "ff0087",
                              "ff00af", "ff00d7", "ff00ff", "ff5f00", "ff5f5f", "ff5f87", "ff5faf", "ff5fd7", "ff5fff", "ff8700", "ff875f", "ff8787", "ff87af", "ff87d7",
                              "ff87ff", "ffaf00", "ffaf5f", "ffaf87", "ffafaf", "ffafd7", "ffafff", "ffd700", "ffd75f", "ffd787", "ffd7af", "ffd7d7", "ffd7ff", "ffff00",
                              "ffff5f", "ffff87", "ffffaf", "ffffd7", "ffffff", "080808", "121212", "1c1c1c", "262626", "303030", "3a3a3a", "444444", "4e4e4e", "585858",
                              "606060", "666666", "767676", "808080", "8a8a8a", "949494", "9e9e9e", "a8a8a8", "b2b2b2", "bcbcbc", "c6c6c6", "d0d0d0", "dadada", "e4e4e4",
                              "eeeeee"]))

#### How fast does this run?

In [ ]:
image = np.random.randint(0, 256, 200*200*3).reshape((200, 200, 3))

In [ ]:
%timeit map_image_pixels(image)

The result is unfortunate, esspecially considering the small size of the image.  
If we wanted to carry this operation out on our whole dataset, we would have a problem.

#### Second Attempt: Vectorizing distance calculation
The distance calculation is simple arithmatic, we can use broadcasting to calculate the distance from a pixel to  
all the others in one shot. For this, we need to convert our color list into a matrix we can use. 

In [ ]:
# color matrix
color_matrix = np.vstack([np.array(c) for c in colors])

Now we adapt the distance calculation function, and the image mapping function to use the vectorized calculation:

In [ ]:
def from_rgb_vec(rgb: list):
    """
    Return the nearest xterm 256 color code from rgb input.
    """
    # convert to ndarray
    c = np.array(rgb)

    # calculate the ditances to each color in color mat
    all_distances = np.sum(np.sqrt((color_matrix-c)**2), axis=1)

    # get the minimum color index
    min_color = np.argmin(all_distances)
    return min_color

def map_image_pixels_vec(image):
    """
    replace all image pixels with the closest xterm color,
    calculates distances with vector arithmatic.
    :param image: ndarray image
    :return: ndarray image
    """
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            new_color = from_rgb_vec(image[i, j])
            image[i, j] = np.array(colors[new_color])
    return image

#### How fast does this run?

In [ ]:
%timeit map_image_pixels_vec(image)

Not bad! 20 fold speedup almost for free. All we did was create a ndarray out of the colors list (pay once per import), and use some simple vector operations. Can we do better?

#### Third Attempt: Calculate for all pixels at one  
Using the same kind of broadcasting we used for each pixel, we can carry out all the distance computation in one shot. This is where broadcasting really becomes interesting. It might be a little hard to grasp at first, so lets do this step by step:

In [ ]:
def map_image_pixels_broad(image):
    """
    replaces all image pixels with closest xterm colors,
    calculates the distances and constructs the new image
    using vector math and broadcasting.
    :param image: ndarray
    :return: ndarray
    """
    # reshape image for ease of broadcasting
    pixel_column = image.reshape(1, image.shape[0]*image.shape[1], 3)

    # move axis for color matrix
    broad_color_mat = color_matrix.reshape(color_matrix.shape[0], 1, 3)

    # distance matrix, broadcasting happens here!
    distance_matrix = np.sum(np.sqrt((broad_color_mat-pixel_column)**2), axis=2)

    # min dist colors
    min_dist_indices = np.argmin(distance_matrix, axis=0)

    # take the pixels from the color mat
    new_pixel_column = np.take(color_matrix, min_dist_indices, axis=0)

    # reshape back to original shape
    new_image = new_pixel_column.reshape((image.shape[0], image.shape[1], 3))
    return new_image

first, lets remove one of the dimensions of the image by turning it into a pixel column. This makes working on it easier, and we can put it back together later. the image shape will be changed from (200, 200, 3) to (200*200, 3).

In [ ]:
pixel_column = image.reshape(image.shape[0]*image.shape[1], 3)
pixel_column.shape

seeing as we want to compute the distance from each image pixel, to each color pixel, we can use the broadcasting rules by adding a dimension of 1 to this pixel column. When we calculate the distance from the colors, the pixel column will be broadcasted along this dimension, and we will get what we are looking for:

In [ ]:
pixel_column = pixel_column[np.newaxis, :, :]
pixel_column.shape

Now lets sort out the colors matrix, we need to add a dimension here as well:

In [ ]:
# move axis for color matrix
broad_color_mat = color_matrix[:, np.newaxis, :]
broad_color_mat.shape

When we subtract these arrays, the broadcasting will cause the given arrays to be "coppied" along the dimensions that match ones. This well given us a subtraction of each image pixel from all color pixels:

In [ ]:
sub_matrix = broad_color_mat-pixel_column
sub_matrix.shape

success! Now we need to raise by the power of 2 and take a square root. Numpy has vectorized functions for this as well. After that, all we need for the distances is to sum across the differences for each of the rgb values, which in our case, is repersented by the 3rd dimensions:

In [ ]:
almost_distance = np.sqrt(sub_matrix**2)
almost_distance.shape

In [ ]:
distance_matrix = np.sum(almost_distance, axis=2)
distance_matrix.shape

Now we want the index of the minimum distance for each image pixel. In matrix-speak, this means taking the minimum index from each column:

In [ ]:
min_dist_indices = np.argmin(distance_matrix, axis=0)

Almost last step, we want to rebuild or image pixel column, taking the corrisponding color pixels from the color matrix. Numpy has a built in function for this as well:

In [ ]:
new_pixel_column = np.take(color_matrix, min_dist_indices, axis=0)

lastly, we reshape the new pixel column to our original image dimensions:

In [ ]:
new_image_broad = new_pixel_column.reshape((image.shape[0], image.shape[1], 3))

how much have we gained by using broadcasting:

In [ ]:
%timeit map_image_pixels_broad(image)

Seems like we get another 20% increase relative to the simple vectorized version!

Lets make sure the results are equivalent throughout the three attempts:

In [ ]:
regular_map = map_image_pixels(image)
vectorized_map = map_image_pixels_vec(image)
broadcast_map = map_image_pixels_broad(image)

In [ ]:
print(f"regular=vectorized: {np.array_equal(regular_map, vectorized_map)}, vectorized=broadcast: {np.array_equal(vectorized_map, broadcast_map)}")

## Case study: Intersection Over Union calculations

When carrying out segmentation or detection tasks, we find ourselves measuring the preformance of our models using the IOU matric. When carried out sequentially, this can be a slow task. Using numpy, we can vectorize many of these operations and reduce running time considerably.

Lets load some data, i'm using the Penn pedastrian dataset, it provides images and masks of pedestrains.

In [ ]:
# lets load a sample image
image_dir = "PennFudanPed/PNGImages"
masks_dir = "PennFudanPed/PedMasks"
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]

@interact(image_id=IntSlider(min=0, max=len(image_paths)-1, step=1, value=0))
def show_image(image_id=0):
    plt.close()
    ped_img = cv2.cvtColor(cv2.imread(image_paths[image_id]), cv2.COLOR_BGR2RGB)
    mask_path = image_paths[image_id].split("/")[-1].split(".")[0] + "_mask.png"
    ped_msk = cv2.imread(os.path.join(masks_dir, mask_path))
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(ped_img)
    ax[1].matshow(np.sum(ped_msk, axis=2))
    plt.show()

Lets grab a random sample, need to convert to RGB, cv2 default is BGR.

In [ ]:
image_id = 32
ped_img = cv2.cvtColor(cv2.imread(image_paths[image_id]), cv2.COLOR_BGR2RGB)
mask_path = image_paths[image_id].split("/")[-1].split(".")[0] + "_mask.png"
ped_msk = cv2.imread(os.path.join(masks_dir, mask_path))

converting the masks into a tensor of binary masks instead of unique pixel value should make things easier. Just for practice, can we do this using a loop? or broadcasting?

#### first try, regular loop:

In [ ]:
# for loop implementation, slightly slower and more verbose
def split_mask_for(mask):
    unq = np.unique(mask)
    mask_mat = np.array(mask[:, :, 0])
    masks = []
    for c in unq:
        masks.append((mask_mat == c))
    return np.dstack(masks).astype(np.int32)

In [ ]:
%timeit split_mask_for(ped_msk)

#### second try, lets use broadcasting:

In [ ]:
# broadcasting trick to split image to mask tensor
def split_mask(mask):
    # unique values in the image + reshape
    uniq = np.unique(mask)
    uniq = uniq[np.newaxis, np.newaxis, :]
    # create 2d mask + reshape
    mask_mat = np.array(mask[:, :, 0])[:, :, np.newaxis]
    # compare and create binary mask
    return (mask_mat == uniq).astype(np.int32)


In [ ]:
%timeit split_mask(ped_msk)

In [ ]:
p = split_mask(ped_msk)
m = split_mask(ped_msk)

#### Native python implementation: Loops only

In [ ]:
def native_iou(pred, masks):
    num_preds = pred.shape[2]
    num_masks = masks.shape[2]
    iou_mat = np.zeros((num_masks, num_preds))
    for pid in range(num_preds):
        for mid in range(num_masks):
            inter = 0
            union = 0
            for row in range(pred.shape[0]):
                for col in range(pred.shape[1]):
                    inter += int(pred[row, col, pid] and masks[row, col, mid])
                    union += int(pred[row, col, pid] or masks[row, col, mid])
            iou_mat[mid, pid] = 0 if (union == 0) else (inter / union)
    return iou_mat

In [ ]:
%timeit native_iou(p, m)

#### First Attempt: Nested Loops, simple array operations  
keep it simple and pythonic, nested loops, calculate intersection and union for each pair and build a matrix.

In [ ]:
def loop_iou(pred, masks):
    # create intersection and union matrices
    intersection = np.zeros((pred.shape[2], masks.shape[2]))
    union = np.zeros_like(intersection)
    
    for i in range(pred.shape[2]):
        for j in range(masks.shape[2]):
            intersection[i,j] = np.logical_and(pred[:,:, i], masks[:, :, j]).sum()
            union[i, j] = np.logical_or(pred[:,:, i], masks[:, :, j]).sum()
    return intersection/union

In [ ]:
%timeit loop_iou(p, m)

#### Second Attempt: save a loop by broadcasting each pred to all masks

In [ ]:
def loop_iou2(pred, masks):
    # create intersection and union matrices
    intersection = np.zeros((pred.shape[2], masks.shape[2]))
    union = np.zeros_like(intersection)    
    iou_vecs = []
    for i in range(pred.shape[2]):
        temp_pred = np.array(pred[:, :, i])[:, :, np.newaxis]  # this looks like its going to slow things down
        intersection = np.logical_and(temp_pred, masks).sum(axis=(0,1))
        union = np.logical_or(temp_pred, masks).sum(axis=(0,1))
        iou_vecs.append(intersection/union)
    return np.vstack(iou_vecs)

In [ ]:
%timeit loop_iou2(p, m)

#### Third Attempt: no loops at all, use only broadcasting.

In [ ]:
def vector_iou(pred, masks):
    print(f"masks: {masks.shape}")
    print(f"pred: {pred.shape}")
    pred = pred[np.newaxis, :]
    masks = np.transpose(masks[np.newaxis, :], (3,1,2,0))
    print(f"masks: {masks.shape}")
    print(f"pred: {pred.shape}")
    intersection = np.sum(np.logical_and(pred, masks), axis=(1, 2))
    union = np.sum(np.logical_or(pred, masks), axis=(1, 2))
    print(f"final: {intersection.shape}")
    return intersection/union